# Using Toolhouse for Tool Use with Together

[Toolhouse](https://app.toolhouse.ai/) is the first complete infrastructure platform for building, running and managing tool use.

With Toolhouse, you can equip your LLM with extra skills (also known as tools).

Some of the most popular tools include:
- scraping data from the web or social media
- generate images
- compile or execute code and return the values

This cookbook demonstrates how you can **equip LLMs running on** [Together.ai](https://together.ai/) - **with tools**, without the need for your to code or prompt these tools.

In this short demo, we'll show how we can equip an LLM to generate images from real-data.

We'll use Toolhouse with the model tagged `meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo` and hosted on Together's infrastructure.

This model is fine-tuned for effective and precise tool use.

### Setup

In [ ]:
!pip install toolhouse openai

In [40]:
# Import packages
import os
from openai import OpenAI
from toolhouse import Toolhouse

To integrate Together and Toolhouse, you'll need to set up two environment variables: `TOGETHER_API_KEY` and `TOOLHOUSE_API_KEY`. Follow these steps to obtain your API keys:

* **Together API Key**: Get your key by visiting the [Together Console](https://api.together.ai/).
* **Toolhouse API Key**: Sign up for Toolhouse using [this link](https://join.toolhouse.ai) to receive $150 in credits. You will receive your API key as part of the onboarding step, and you can always, navigate to the [Toolhouse API Keys page](https://app.toolhouse.ai/settings/api-keys) to create and get an API key.
* **Install the X tool**: In your [Toolhouse dashboard](https://app.toolhouse.ai), click Install next to the "Search X" tool ([direct link](https://app.toolhouse.ai/store/search_x)).
* **Install the Image Generation tool**: In your [Toolhouse dashboard](https://app.toolhouse.ai), click Install next to the "Image Generation" tool ([direct link](https://app.toolhouse.ai/store/image_generation_flux)).

Once you have both API keys, set them as environment variables to start using Together and Toolhouse.

To do that click on the Key icon in the left menu on Colab or define them in your environment variables and use them with `os.getenv()`.



In [ ]:
tools=th.get_tools("weather")

messages = [
    {"role": "system", "content": "You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls."},
    {"role": "user", "content": "What is the current temperature of New York?"}
]

response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

messages += th.run_tools(response)

final_response = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

print(final_response.choices[0].message.content)


# if tool_calls:
#     for tool_call in tool_calls:
#         function_name = tool_call.function.name
#         function_args = json.loads(tool_call.function.arguments)

#         if function_name == "get_current_weather":
#             function_response = get_current_weather(
#                 location=function_args.get("location"),
#                 unit=function_args.get("unit"),
#             )
#             messages.append(
#                 {
#                     "tool_call_id": tool_call.id,
#                     "role": "tool",
#                     "name": function_name,
#                     "content": function_response,
#                 }
#             )

#     function_enriched_response = client.chat.completions.create(
#         model="mistralai/Mixtral-8x7B-Instruct-v0.1",
#         messages=messages,
#     )
#     print(json.dumps(function_enriched_response.choices[0].message.model_dump(), indent=2))

None


We've imported the SDKs, now let's initialize them and authenticate ourselves using an API key. One per service respectively. Your API keys can be found in the left sidebar of Google Colab after clicking the key icon.

In [46]:
from google.colab import userdata
client = OpenAI(base_url = "https://api.together.xyz/v1", api_key=userdata.get("TOGETHER_API_KEY"))

th = Toolhouse(api_key=userdata.get('TOOLHOUSE_API_KEY'), provider="openai")

We will use a Meta model specificially the Instruct flavor:
- [meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
 model](https://docs.together.ai/docs/function-calling#supported-models).

This model is based on the Meta 3.1 70B model and was fine-tuned for tool use and function calling:

We can test that we can call the Mixtral model hosted on Together by running the cell below. It should output some info about NYC.

In [74]:
MODEL = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "tell me about new york"}],
)
print(response.choices[0].message.content)


New York, also known as the "Empire State," is a state located in the northeastern United States. It is the 27th largest state by area and the 4th most populous state in the country. Here are some interesting facts and attractions about New York:

**Cities:**

1. New York City: The most populous city in the United States, known for its iconic skyline, Broadway shows, and world-class museums like the Metropolitan Museum of Art and the Museum of Modern Art (MoMA).
2. Buffalo: Located in western New York, known for its rich history, cultural attractions, and natural beauty, including Niagara Falls.
3. Albany: The capital city of New York, known for its historic architecture, cultural attractions, and the New York State Capitol building.
4. Rochester: Located in western New York, known for its rich history, cultural attractions, and the George Eastman Museum.

**Attractions:**

1. Statue of Liberty: A iconic symbol of freedom and democracy, located in New York Harbor.
2. Central Park: A la

Likewise, you can use the `th.get_tools()` function to display all of the Toolhouse tools you have installed:

In [75]:
print('TOOLS AVAILABLE:')
for tool in th.get_tools("together"):
    print(f"Name: {tool['function']['name']}")
    print(f"Type: {tool['type']}")
    print(f"Description: {tool['function']['description']}")

TOOLS AVAILABLE:
Name: search_x
Type: function
Description: This tool helps your LLM refine and target specific tweets (or posts) from X (formerly Twitter) database. By using advanced search operators, the LLM can compose precise search queries to locate the most relevant posts based on criteria like user, date, media type, and more.
Name: image_generation_flux
Type: function
Description: This tools enables the LLM to generate images from a given description. The description comes in the format of a prompt. For example: `a photo of a cat, sitting on the couch, looking at the camera, hyper realistic, ultra high definition`. The image generated is returned to the LLM as a JSON containing a string which represents the image in an embeddable markdown format. Only return the markdown embed code to the user. For example: ![](https://example.com/image/url-string)


For this demo, we will be using two tools from Toolhouse's store.
- Search X
- Image generation

The first tool lets you search the X social network and the second tool lets you generate an image from a prompt and returns it to the LLM.

### Configure Tool Calls

First we'll configure the user message to the LLM with the code we want run:

In [101]:
# User message to the LLM
messages = [
  {
    "role": "user",
    "content": "Find the last three messages from the account named @togethercompute on X/Twitter and summarize them in one sentence. Make it funny!",
  }
]

[{'role': 'user', 'content': 'Find the last three messages from the account named @togethercompute on X/Twitter and summarize them in one sentence. Make it funny!'}]


We'll send this to our LLM hosted on TogetherAI. This allows you to skip having to code your own Search X parser since we are using the tool hosted and maintained by the Toolhouse team:

In [102]:
# TogetherAI response with Toolhouse tools
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes the tools to the model
  tools=th.get_tools("together"),
)

print(response.choices[0].message.tool_calls[0].id)

call_ynb9zloi1bp7012c28n36a01


As you can see from the output above, the LLM properly identified that we'd like to invoke the 'search_x' tool

You can see the arguments it generated by running the code below, it will break iut down by ID, type and function arguments:

In [103]:
tools_called = response.choices[0].message.tool_calls
for tool_called in tools_called:
    print(f"ID: {tool_called.id}")
    print(f"Type: {tool_called.type}")
    print(f"Function: {tool_called.function}")
    print('\n')

ID: call_ynb9zloi1bp7012c28n36a01
Type: function
Function: Function(arguments='{"query":"from:togethercompute","max_results":"3"}', name='search_x')




### Execute Tool Call

Now that we learned how the LLM can generate arguments to pass to a function we'll have to **run the function with those arguments**.

The tool gets executed via the `run_tools` command, with the parameters that were identified in the previous LLM call.


After that we will get the result, and append it to the context, the `messages` variable.

In [124]:
tool_run = th.run_tools(response)
messages += tool_run

ChatCompletion(id='8e0b248b1bea8832-ATL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ynb9zloi1bp7012c28n36a01', function=Function(arguments='{"query":"from:togethercompute","max_results":"3"}', name='search_x'), type='function', index=0)]), seed=14357862309395075000)], created=1731295187, model='meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=25, prompt_tokens=860, total_tokens=885, completion_tokens_details=None, prompt_tokens_details=None), prompt=[])


Let's see what our messages list looks like:

In [105]:
for message in tool_run:
  print(f"Role: {message['role']}")
  if 'tool_calls' in message:
    print(f"Tool Calls: {message['tool_calls']}")
  if 'tool_call_id' in message:
    print(f"Tool Call ID: {message['tool_call_id']}")
    print(f"Content: {message['content']}")
    print('\n')

Role: assistant
Tool Calls: [{'id': 'call_ynb9zloi1bp7012c28n36a01', 'function': {'arguments': '{"query":"from:togethercompute","max_results":"3"}', 'name': 'search_x'}, 'type': 'function', 'index': 0}]
Role: tool
Tool Call ID: call_ynb9zloi1bp7012c28n36a01
Content: <tweet>https://t.co/5OI0Gmyb7Y
by @togethercompute at 2024-11-08T17:21:22.000Z
 Conversation ID: 1854937267552272638

![github.com/togethercomput…](https://github.com/togethercomputer/together-cookbook/blob/main/Summarization_Evaluation.ipynb
<tweet>

<tweet>New Cookbook: Summarizing and Evaluating Outputs with LLMs.

In this notebook we:
1. Generate summaries using Llama 3.1 70B
2. Assess summary quality using BERTScore evaluated between original document chunks and the summary.

Link Below: https://t.co/PWL3vPY1GU
by @togethercompute at 2024-11-08T17:21:22.000Z
 Conversation ID: 1854937267552272638


Photo: ![Image](https://pbs.twimg.com/media/Gb4PSoQWEAAov31.jpg)<tweet>

<tweet>https://t.co/8ZBdRFnRPv
by @togethercompute

We're now going to pass the whole message chain to the LLM. The LLM will generate a proper response:

In [106]:
summary_response = client.chat.completions.create(
  model=MODEL,
  messages=messages
)

print('LLM RESPONSE:', summary_response.choices[0].message.content)

LLM RESPONSE: Here's a funny summary of the last three messages from @togethercompute: "Together Compute is on a roll, releasing new guides on implementing Contextual RAG and improving semantic search, while also sharing a recording of a talk on building LlamaCoder, because who doesn't love a good AI app?"


Let's add this last response - a funny summary of three posts retrieved from X - to the history of messages.

Then let's use another tool to generate an image.

In [131]:
new_messages = messages + [
  {
    "role": "assistant",
    "content": summary_response.choices[0].message.content,
  }
]

LLM RESPONSE: [{'role': 'user', 'content': 'Find the last three messages from the account named @togethercompute on X/Twitter and summarize them in one sentence. Make it funny!'}, {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'tool_calls': [{'id': 'call_ynb9zloi1bp7012c28n36a01', 'function': {'arguments': '{"query":"from:togethercompute","max_results":"3"}', 'name': 'search_x'}, 'type': 'function', 'index': 0}]}, {'role': 'tool', 'tool_call_id': 'call_ynb9zloi1bp7012c28n36a01', 'name': 'search_x', 'content': "<tweet>https://t.co/5OI0Gmyb7Y\nby @togethercompute at 2024-11-08T17:21:22.000Z\n Conversation ID: 1854937267552272638\n\n![github.com/togethercomput…](https://github.com/togethercomputer/together-cookbook/blob/main/Summarization_Evaluation.ipynb\n<tweet>\n\n<tweet>New Cookbook: Summarizing and Evaluating Outputs with LLMs.\n\nIn this notebook we:\n1. Generate summaries using Llama 3.1 70B\n2. Assess summary quality using BERTScore evaluated between origin

Next we're going to give a new command to the LLM - generate us an image from this funny caption using the tool provided by Toolhouse.

This tool uses Flux to take a prompt and generates an image, then hosts it  to storage on the img.toolhouse.ai subdomain.

In [144]:

generate_image_request = new_messages + [
  {
    "role": "user",
    "content": "Generate an image from the funny summary"
  }
]

# Let's ask the LLM to generate the image
image_response = client.chat.completions.create(
  model=MODEL,
  messages=generate_image_request,
  tools=th.get_tools(),
)
tool_run = th.run_tools(image_response)

KeyboardInterrupt: 

Now that the LLM generated an image and stored it, we just want it to let the user know that the image can be viewed. So we're going to yet again ask the LLM to take the output of the function call and generate an image.

In [143]:
image_messages = generate_image_request + tool_run

last_response = client.chat.completions.create(
  model=MODEL,
  messages=image_messages,
  tools=th.get_tools(),
)

print('LLM RESPONSE:', last_response.choices[0].message.content)


LLM RESPONSE: Here is the image from the funny summary: ![](https://img.toolhouse.ai/w0CBb3.jpg)


And there you have it, if you have managed to run all the cells in the correct order you should see a message with a valid URL. If you click it you will be able to see the generated image!